# Termination

In the previous section, we explored how to define agents, and organize them into teams that can solve tasks. However, a run can go on forever, and in many cases, we need to know when to stop them. This is the role of the termination condition.

AgentChat supports several termination condition by providing a base [TerminationCondition](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition) class and several implementations that inherit from it.

A termination condition is a callable that takes a sequence of [BaseAgentEvent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.BaseAgentEvent) or [BaseChatMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.BaseChatMessage) objects **since the last time the condition was called**, and returns a [StopMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.StopMessage) if the conversation should be terminated, or `None` otherwise. Once a termination condition has been reached, it must be reset by calling [reset()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition.reset) before it can be used again.

Some important things to note about termination conditions:

- They are stateful but reset automatically after each run (`run()` or `run_stream()`) is finished.

- They can be combined using the AND and OR operators.

**⚠️ Note**: *For group chat teams (i.e., [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat), [SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat), and [Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm)), the termination condition is called after each agent responds. While a response may contain multiple inner messages, the team calls its termination condition just once for all the messages from a single response. So the condition is called with the “delta sequence” of messages since the last time it was called.*

Built-In Termination Conditions:

1. [MaxMessageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.MaxMessageTermination): Stops after a specified number of messages have been produced, including both agent and task messages.

2. [TextMentionTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMentionTermination): Stops when specific text or string is mentioned in a message (e.g., “TERMINATE”).

3. [TokenUsageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TokenUsageTermination): Stops when a certain number of prompt or completion tokens are used. This requires the agents to report token usage in their messages.

4. [TimeoutTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TimeoutTermination): Stops after a specified duration in seconds.

5. [HandoffTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.HandoffTermination): Stops when a handoff to a specific target is requested. Handoff messages can be used to build patterns such as [Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm). This is useful when you want to pause the run and allow application or user to provide input when an agent hands off to them.

6. [SourceMatchTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.SourceMatchTermination): Stops after a specific agent responds.

7. [ExternalTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.ExternalTermination): Enables programmatic control of termination from outside the run. This is useful for UI integration (e.g., “Stop” buttons in chat interfaces).

8. [StopMessageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.StopMessageTermination): Stops when a [StopMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.StopMessage) is produced by an agent.

9. [TextMessageTermination](): Stops when a [TextMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.TextMessage) is produced by an agent.

10. [FunctionCallTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.FunctionCallTermination): Stops when a [ToolCallExecutionEvent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.ToolCallExecutionEvent) containing a [FunctionExecutionResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.models.html#autogen_core.models.FunctionExecutionResult) with a matching name is produced by an agent.

11. [FunctionalTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.FunctionalTermination): Stop when a function expression is evaluated to `True` on the last delta sequence of messages. This is useful for quickly create custom termination conditions that are not covered by the built-in ones.


## Basic Usage

To demonstrate the characteristics of termination conditions, we’ll create a team consisting of two agents: a primary agent responsible for text generation and a critic agent that reviews and provides feedback on the generated text.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-3.5-turbo",
    temperature=1,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- TextMessage (primary) ----------
Raindrops gently fall,
Eiffel Tower shrouded in mist,
Paris whispers peace.
---------- TextMessage (critic) ----------
I love the imagery and peaceful tone in your haiku! It captures the essence of a rainy day in Paris beautifully. Well done!


TaskResult(messages=[TextMessage(id='2e22adaf-2838-47ec-ab18-acf71eba3f5a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 58, 10, 428088, tzinfo=datetime.timezone.utc), content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(id='52ca812d-e962-4ea4-a28d-54d70646c1d8', source='primary', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 58, 11, 732402, tzinfo=datetime.timezone.utc), content='Raindrops gently fall,\nEiffel Tower shrouded in mist,\nParis whispers peace.', type='TextMessage'), TextMessage(id='82c19b5c-c7d1-438f-af6d-7c91fa13c1ff', source='critic', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 58, 12, 631878, tzinfo=datetime.timezone.utc), content='I love the imagery and peaceful tone in your haiku! It captures the essence of a rainy day in Pa

The conversation stopped after reaching the maximum message limit. Since the primary agent didn’t get to respond to the feedback, let’s continue the conversation.

In [4]:
# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream())

---------- TextMessage (primary) ----------
Thank you! I'm glad you enjoyed the Haiku. Let me know if there's anything else I can help you with!
---------- TextMessage (critic) ----------
Your haiku was lovely and well-crafted. It was a pleasure to read! Keep up the great work.
---------- TextMessage (primary) ----------
Thank you for your kind words! I'm here to assist you whenever you need. Just let me know how I can help!


TaskResult(messages=[TextMessage(id='603377a7-3cd7-47ca-9a44-be3411927b51', source='primary', models_usage=RequestUsage(prompt_tokens=87, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 59, 34, 62591, tzinfo=datetime.timezone.utc), content="Thank you! I'm glad you enjoyed the Haiku. Let me know if there's anything else I can help you with!", type='TextMessage'), TextMessage(id='2a02c38a-8b04-4959-9317-c05772b5ff4d', source='critic', models_usage=RequestUsage(prompt_tokens=133, completion_tokens=23), metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 59, 35, 185480, tzinfo=datetime.timezone.utc), content='Your haiku was lovely and well-crafted. It was a pleasure to read! Keep up the great work.', type='TextMessage'), TextMessage(id='765bd989-b6ea-44be-b34e-770047442268', source='primary', models_usage=RequestUsage(prompt_tokens=147, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 7, 28, 4, 59, 35, 975883, tzinfo=datetime.tim

The team continued from where it left off, allowing the primary agent to respond to the feedback.

## Combining Termination Conditions

Let’s show how termination conditions can be combined using the AND (`&`) and OR (`|`) operators to create more complex termination logic. For example, we’ll create a team that stops either after 10 messages are generated or when the critic agent approves a message.

In [5]:
max_msg_termination = MaxMessageTermination(max_messages=10)
text_termination = TextMentionTermination("APPROVE")
combined_termination = max_msg_termination | text_termination

round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))

---------- TextMessage (user) ----------
Write a unique, Haiku about the weather in Paris
---------- TextMessage (primary) ----------
Cool breeze through the streets,
Sunset paints the sky with gold,
Paris dreams in light.
---------- TextMessage (critic) ----------
I appreciate the vivid imagery and serene atmosphere in your haiku. It effectively captures the tranquility of a Parisian evening. Well done!
---------- TextMessage (primary) ----------
Thank you for your feedback! I'm glad you enjoyed the haiku. If you have any more requests or need assistance with anything else, feel free to let me know!
---------- TextMessage (critic) ----------
Your haiku beautifully depicts the essence of a Parisian evening. Keep up the great work! If you have any other haiku or creative writing pieces, feel free to share them. Great job!
---------- TextMessage (primary) ----------
Thank you for your encouraging words! I'm glad you liked the haiku. If you have any other preferences or topics you'd like 

TaskResult(messages=[TextMessage(id='6c620ef9-f6bb-407a-83b8-94324cc997b5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 28, 5, 0, 52, 957224, tzinfo=datetime.timezone.utc), content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(id='6bbf392b-9f31-4e4d-be53-f1834c9d0eb0', source='primary', models_usage=RequestUsage(prompt_tokens=193, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 28, 5, 0, 53, 901192, tzinfo=datetime.timezone.utc), content='Cool breeze through the streets,\nSunset paints the sky with gold,\nParis dreams in light.', type='TextMessage'), TextMessage(id='855a9e30-67c2-402e-91d5-c6e30871ac23', source='critic', models_usage=RequestUsage(prompt_tokens=201, completion_tokens=28), metadata={}, created_at=datetime.datetime(2025, 7, 28, 5, 0, 54, 751597, tzinfo=datetime.timezone.utc), content='I appreciate the vivid imagery and serene atmosphere in your haiku. It effectively c

The conversation stopped after the critic agent approved the message, although it could have also stopped if 10 messages were generated.

Alternatively, if we want to stop the run only when both conditions are met, we can use the AND (&) operator.

```python
combined_termination = max_msg_termination & text_termination
```

## Custom Termination Condition

The built-in termination conditions are sufficient for most use cases. However, there may be cases where you need to implement a custom termination condition that doesn’t fit into the existing ones. You can do this by subclassing the [TerminationCondition](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition) class.

In this example, we create a custom termination condition that stops the conversation when a specific function call is made.

In [ ]:
from typing import Sequence

from autogen_agentchat.base import TerminatedException, TerminationCondition
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage, StopMessage, ToolCallExecutionEvent
from autogen_core import Component
from pydantic import BaseModel
from typing_extensions import Self


class FunctionCallTerminationConfig(BaseModel):
    """Configuration for the termination condition to allow for serialization
    and deserialization of the component.
    """

    function_name: str


class FunctionCallTermination(TerminationCondition, Component[FunctionCallTerminationConfig]):
    """Terminate the conversation if a FunctionExecutionResult with a specific name is received."""

    component_config_schema = FunctionCallTerminationConfig
    component_provider_override = "autogen_agentchat.conditions.FunctionCallTermination"
    """The schema for the component configuration."""

    def __init__(self, function_name: str) -> None:
        self._terminated = False
        self._function_name = function_name

    @property
    def terminated(self) -> bool:
        return self._terminated

    async def __call__(self, messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> StopMessage | None:
        if self._terminated:
            raise TerminatedException("Termination condition has already been reached")
        for message in messages:
            if isinstance(message, ToolCallExecutionEvent):
                for execution in message.content:
                    if execution.name == self._function_name:
                        self._terminated = True
                        return StopMessage(
                            content=f"Function '{self._function_name}' was executed.",
                            source="FunctionCallTermination",
                        )
        return None

    async def reset(self) -> None:
        self._terminated = False

    def _to_config(self) -> FunctionCallTerminationConfig:
        return FunctionCallTerminationConfig(
            function_name=self._function_name,
        )

    @classmethod
    def _from_config(cls, config: FunctionCallTerminationConfig) -> Self:
        return cls(
            function_name=config.function_name,
        )

Let’s use this new termination condition to stop the conversation when the critic agent approves a message using the `approve` function call.

First we create a simple function that will be called when the critic agent approves a message.

In [ ]:
def approve() -> None:
    """Approve the message when all feedbacks have been addressed."""
    pass

Then we create the agents. The critic agent is equipped with the approve tool.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=1,
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent with the approve function as a tool.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    tools=[approve],  # Register the approve function as a tool.
    system_message="Provide constructive feedback. Use the approve tool to approve when all feedbacks are addressed.",
)

Now, we create the termination condition and the team. We run the team with the poem-writing task.

In [ ]:
function_call_termination = FunctionCallTermination(function_name="approve")
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=function_call_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"))
await model_client.close()

You can see that the conversation stopped when the critic agent approved the message using the `approve` function call.